In [1]:
from Agents.utils import split_notebook_turns
from Agents.agents import turn_classifier
import json
from langchain_core.messages import FunctionMessage, HumanMessage

{"detail":"User is not allowed to create run. UserProjectRole with User Email 'daniel.oluwakayode-c@turing.com', Engagement Name 'public-engagement' and Project Name 'beta-test' not found"}
Authentication failed.


In [2]:
conversation_turns = split_notebook_turns("Data/python_basics_&_scripting__write_code_in_python__22__24_03_2024_08_57_45_10.ipynb")

In [4]:
conversation_turns[1]

[{'role': 'Assistant',
  'content': "Sure, I can help you with that. Here's how you can implement seek and write operations to modify a specific record in a binary file containing employee data in Python:\n\n**Approaches**:\n\n1. **Calculate Byte Offset**: Determine the starting byte position of the target record based on the record size and record number.\n\n2. **Open File in 'rb+ mode**: Open the binary file in read-write binary mode (`'rb+'`). This allows both reading and modifying the file contents.\n\n3. **Seek to Record Position**: Use the `file.seek()` method to move the file pointer to the calculated byte offset of the target record.\n\n4. **Read Existing ID (Optional)**: You can optionally read the existing ID at this position for verification purposes using `file.read(size)`.\n\n5. **Encode New Name**: Encode the new name string with the same fixed length as the original name field (truncate or pad with spaces).\n\n6. **Write New Name**: Use `file.write()` to write the encode

In [7]:
result = conversation_turns[0]
result = turn_classifier([HumanMessage("hello how are you?")])
print(result)
all([result["has_code"], result["complete_code"], result["can_be_tested"]])

Validating output schema.....
{'has_code': False, 'complete_code': False, 'can_be_tested': False, 'code': '', 'dependencies': [], 'language': ''}


False

In [6]:
result = turn_classifier([HumanMessage("print('i am fine)")])
print(result)


Validating output schema.....
{'has_code': True, 'complete_code': False, 'can_be_tested': False, 'code': "print('i am fine)", 'dependencies': [], 'language': 'python3'}
